# 2023 Day 8

https://adventofcode.com/2023/day/8

https://adventofcode.com/2023/day/8/input

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
inp = open('input-08.txt').read().strip()
# print(inp)

## Part 1

In [3]:
def parse_branches(para):
    mapping = {}
    lines = para.split('\n')
    for line in lines:
        a, b, c = re.match(r'(\w{3}) = \((\w{3}), (\w{3})\)', line).groups()
        mapping[a] = (b, c)
    return mapping

def parse_map(text):
    paras = text.split('\n\n')
    directions = paras[0]
    branches = parse_branches(paras[1])
    return directions, branches

In [4]:
directions, branches = parse_map(inp)
# directions

In [5]:
# branches

In [6]:
i_dir = 0
s = 'AAA'
while s != 'ZZZ':
    direction = 0 if directions[i_dir % len(directions)] == 'L' else 1
    s = branches[s][direction]
    i_dir += 1
i_dir

17263

## Part 2

In [7]:
starts = [k for k in branches if k.endswith('A')]
starts

['GCA', 'AAA', 'CMA', 'QNA', 'FTA', 'CBA']

In [8]:
ends = [k for k in branches if k.endswith('Z')]
ends

['SMZ', 'FCZ', 'TBZ', 'TTZ', 'XTZ', 'ZZZ']

In [9]:
def when_return_when_z(s):
    # I thought I'd determine when we get back to the start
    # and note each "ends-in-z" index along the way
    s0 = s
    i_dir = 0
    # also keep track of which ends-in-z we land on
    which_z = []
    when_z = []
    while s != s0 or i_dir == 0:
        if s.endswith('Z'):
            when_z.append(i_dir)
            which_z.append(s)
        direction = 0 if directions[i_dir % len(directions)] == 'L' else 1
        s = branches[s][direction]
        i_dir += 1
        # this is taking too long, return what you got so far
        if i_dir > 1_000_000:
            break
    return i_dir, when_z, which_z

In [10]:
# apparently we actually start to cycle as soon as we hit an ends-in-z
# which means we never return to the start, but always return to the same ends-in-z
# ... which means our strategy of finding loop lengths still works
rates = []
for s in starts:
    i, when, which = when_return_when_z(s)
    rate = when[1] - when[0]
    rates.append(rate)
    print(s, when[:4], np.unique(which), np.diff(np.r_[0, when[:5]]), rate)

GCA [22357, 44714, 67071, 89428] ['FCZ'] [22357 22357 22357 22357 22357] 22357
AAA [17263, 34526, 51789, 69052] ['ZZZ'] [17263 17263 17263 17263 17263] 17263
CMA [14999, 29998, 44997, 59996] ['SMZ'] [14999 14999 14999 14999 14999] 14999
QNA [16697, 33394, 50091, 66788] ['TTZ'] [16697 16697 16697 16697 16697] 16697
FTA [13301, 26602, 39903, 53204] ['TBZ'] [13301 13301 13301 13301 13301] 13301
CBA [20659, 41318, 61977, 82636] ['XTZ'] [20659 20659 20659 20659 20659] 20659


In [11]:
# now we just need to find when all loops align at their ends-in-z points
m = 1
for rate in rates:
    # least common multiple
    m = np.lcm(m, rate)
m

14631604759649